In [1]:
import pandas as pd
import itertools
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor, Pool

In [2]:
def evaluate_all_combinations_cv_catboost(X, y):
    results = []
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    for r in range(1, len(X.columns) + 1):
        for combo in itertools.combinations(X.columns, r):
            X_subset = X[list(combo)]
            model = CatBoostRegressor(
                iterations=500,
                learning_rate=0.1,
                depth=6,
                random_state=42,
                verbose=0
            )
            scores = cross_val_score(model, X_subset, y, cv=kf, scoring='r2')
            mean_r2 = scores.mean()
            results.append({
                'num_features': r,
                'features': combo,
                'mean_r2': mean_r2
            })

    results_df = pd.DataFrame(results).sort_values(by='mean_r2', ascending=False).reset_index(drop=True)

    for i, row in results_df.iterrows():
        print(f"{i+1}. Features: {row['features']}, R²: {row['mean_r2']:.4f}")

    best = results_df.iloc[0]
    return best['features'], best['mean_r2'], results_df

In [3]:
df = pd.read_csv('dataset.csv')
X = df[['cement', 'blast furnace slag', 'fly ash', 'water ', 'superplasticizer', 'coarse aggregate', 'fine aggregate', 'age']]
y = df['CCS']

# CatBoost로 조합 평가
best_combo, best_r2, all_results = evaluate_all_combinations_cv_catboost(X, y)

print("\n📌 최고 조합:", best_combo)
print("📈 최고 평균 결정계수(R²):", best_r2)

1. Features: ('cement', 'blast furnace slag', 'fly ash', 'water ', 'superplasticizer', 'coarse aggregate', 'fine aggregate', 'age'), R²: 0.9416
2. Features: ('cement', 'blast furnace slag', 'fly ash', 'water ', 'superplasticizer', 'fine aggregate', 'age'), R²: 0.9412
3. Features: ('cement', 'blast furnace slag', 'water ', 'superplasticizer', 'coarse aggregate', 'fine aggregate', 'age'), R²: 0.9406
4. Features: ('cement', 'blast furnace slag', 'fly ash', 'water ', 'fine aggregate', 'age'), R²: 0.9406
5. Features: ('cement', 'blast furnace slag', 'water ', 'coarse aggregate', 'fine aggregate', 'age'), R²: 0.9393
6. Features: ('cement', 'blast furnace slag', 'fly ash', 'water ', 'superplasticizer', 'coarse aggregate', 'age'), R²: 0.9392
7. Features: ('cement', 'blast furnace slag', 'water ', 'superplasticizer', 'fine aggregate', 'age'), R²: 0.9391
8. Features: ('cement', 'blast furnace slag', 'fly ash', 'water ', 'coarse aggregate', 'fine aggregate', 'age'), R²: 0.9390
9. Features: ('ceme